In [10]:
from __future__ import print_function
#import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

import os

# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[1:], 'тренировочные примеры')
print(x_test.shape[1:], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
(32, 32, 3) тренировочные примеры
(32, 32, 3) тестовые примеры


In [11]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


print('Не используется data augmentation')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Не используется data augmentation
Train on 50000 samples, validate on 10000 samples
50000/50000 [==============================] - 5636s 113ms/sample - loss: 1.9544 - acc: 0.2807 - val_loss: 1.6200 - val_acc: 0.4054
сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 265s 27ms/sample - loss: 1.6200 - acc: 0.4054
Test loss: 1.6199910636901855
Test accuracy: 0.4054


Вывод: точность получившейся модели - 0.4054. Сначала испугался, что низкая точность, но потом вспомнил, что у нас не бинарная, а многоклассовая классификация из 10 классов. Если крутить рулетку, то точность такого случайного классификатора получится = 0.1. Получившийся классификатор в 4 раза лучше случайного. Для увеличения точность можно попробовать увеличить кол-во эпох.  